In [1]:
import os
import sys
import math
from dataclasses import dataclass
from pathlib import Path

import numpy as np
import pandas as pd
import torch
import torch.nn.functional as F
from natsort import natsorted
from safetensors.torch import load_model
from tqdm import tqdm

from models import hvatnet
from utils import creating_dataset, inference_utils
from utils.creating_dataset import LEFT_TO_RIGHT_HAND

sys.path.insert(1, os.path.realpath(os.path.pardir))


In [5]:
## Init model
device = 'cuda:0'
dtype = torch.float32

MODEL_TYPE = 'hvatnet'
weights = r"logs\test_2_run_fedya\step_1200_loss_0.2587.safetensors"
model_config = hvatnet.Config(n_electrodes=8, n_channels_out=20,
                            n_res_blocks=3, n_blocks_per_layer=3,
                            n_filters=128, kernel_size=3,
                            strides=(2, 2, 2), dilation=2, 
                            small_strides = (2, 2))
model = hvatnet.HVATNetv3(model_config)

load_model(model, weights)

model = model.to(device).to(dtype)

Number of parameters: 4210788


In [6]:
amputee_path = r"D:\Work\alvi_labs\code\data\processed\dataset_v2_blocks\amputant\left\fedya_tropin_standart_elbow_left\preproc_angles"
data_folder = Path(amputee_path) / 'submit'
all_paths = natsorted(data_folder.glob('*.npz'))

pred_list = []

for p in all_paths:
    sample = np.load(p)
    myo = sample['data_myo']
    myo = myo[:, LEFT_TO_RIGHT_HAND]
    gt_len = myo[::8].shape[0]

    # padding
    target_length = (myo.shape[0] + 255) // 256 * 256
    padded_myo = np.pad(myo, ((0, target_length - myo.shape[0]), (0, 0)), mode='constant', constant_values=0)

    # some prediction. might be slididng window.
    preds = model.inference(padded_myo)
    
    preds = preds[:gt_len]
    pred_list.append(preds)

pred_cat = np.concatenate(pred_list, axis=0)
df = pd.DataFrame(pred_cat)
df.to_csv('submit_file.csv', index=False)

### Get metrics

In [7]:
submit_file_path = r"D:\Work\alvi_labs\code\data\processed\dataset_v2_blocks\amputant\left\fedya_tropin_standart_elbow_left\preproc_angles\submit_gt.csv"

df_pred = pd.read_csv('submit_file.csv').to_numpy()
df_gt = pd.read_csv(submit_file_path).to_numpy()

angles, corrs = inference_utils.calculate_angle_metrics([df_pred], [df_gt])
angles, corrs

([15.831228825960016], [0.7604160151343241])